In [1]:
from pynq.overlays.base import BaseOverlay
import time
from datetime import datetime
base = BaseOverlay("base.bit")
btns = base.btns_gpio

In [2]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
int write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    return 0;
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

void clear_gpios(){
    for(int i = 0; i < 8; ++i){
        write_gpio(i,0);
    }
}

In [11]:
clear_gpios()

In [14]:
def write_gpio_pwm(pin, brightness, duration=0.5, freq=200):
    """
    pin: RBG -> pins 2, 1, 3 respectively
    brightness: 0–100
    duration: time LED stays on (s)
    freq: PWM frequency (Hz)
    """
    if brightness < 0: brightness = 0
    if brightness > 100: brightness = 100

    period = 1.0 / freq
    on_time = period * (brightness / 100.0)
    off_time = period - on_time

    end_time = time.time() + duration

    while time.time() < end_time:
        write_gpio(pin, 1)
        time.sleep(on_time)
        write_gpio(pin, 0)
        time.sleep(off_time)
    write_gpio(pin, 0)

In [22]:
write_gpio_pwm(1,50, duration=2)